<a href="https://colab.research.google.com/github/GoverdhanReddyGarudaiah/TensorFlow_Practice/blob/main/NLP_SupportTicketCategorization_GoverdhanReddyG_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing and Importing libraries

In [1]:
import pandas as pd
import numpy as np
import re

# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

## Data Overview

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/AIML/Support_ticket_text_data_mid_term.csv")

In [ ]:
df.head()

In [ ]:
df.isnull().sum()



1.   There are two features in the data set 1. Support Ticket ID 2. Support Ticket Text
2.   There are no null values in any of the features



## Model building

In [ ]:
#Installation for GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir -q

In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_M.gguf"
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
    )

In [ ]:
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2,
    n_batch=512,
    n_gpu_layers=43,
    n_ctx=4096,
)

This function would accept Support ticket as one input parameter and System Message to specify the task as second paramter.

In [ ]:
def generateLLAMAPrompt(SupportText, SystemMessage, MaxTokens, Temp):
  system_message = f"""[INST]<<SYS>>{SystemMessage}"<</SYS>>[/INST]"""
  prompt = f"{SupportText}\n{system_message}"
  response = lcpp_llm(
        prompt=prompt,
        max_tokens=MaxTokens,
        temperature=Temp,
        top_p=0.15,
        repeat_penalty=1.2,
        top_k=50,
        stop=['INST'],
        echo=False
    )
  response_text = response["choices"][0]["text"]
  return response_text

## Task 1 - Ticket Categorization

In [ ]:
QueryQuestionTicketCategorization = "Based on the information provided, the category of this support ticket is: .Return category only"

In [ ]:
def getTicketCategoryFromModelResponse(CurrentText):
    LLAMA_Response = generateLLAMAPrompt(CurrentText, QueryQuestionTicketCategorization)
    colon_index = LLAMA_Response.find(':')
    if colon_index != -1:
        category = LLAMA_Response[colon_index+1:].strip()
        pattern = r'[*":.]'
        category = re.sub(pattern, '', category)
        return category.replace('Category','')
    else:
        return None

In [ ]:
df['ticket_category'] = df['support_ticket_text'].apply(lambda x: getTicketCategoryFromModelResponse(x))

In [ ]:
df['ticket_category']

## Task 2 - Creating Tags

In [ ]:
QueryQuestionCreateTags = "Based on the information provided, what are the tags for this support ticket: .Return tags only"

In [ ]:
def getTicketTagsFromModelResponse(CurrentText):
    LLAMA_Response = generateLLAMAPrompt(CurrentText, QueryQuestionCreateTags, 100, 0.01)
    print(LLAMA_Response)
    colon_index = LLAMA_Response.find(':')
    if colon_index != -1:
        category = LLAMA_Response[colon_index+1:].strip()
        pattern = r'[*":.]'
        category = re.sub(pattern, '', category)
        return category.replace('Category','')
    else:
        return None

In [ ]:
qry = "I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?"

In [ ]:
getTicketTagsFromModelResponse(qry)

In [ ]:
getTicketTagsFromModelResponse("Urgent help required! My laptop refuses to start, and I have a crucial presentation scheduled for tomorrow. I've attempted a restart, but it hasn't worked. Please provide immediate assistance to resolve this hardware issue")

## Task 3 - Assigning Priority and ETA

In [ ]:
QueryQuestionAssignPriority = "Based on the information provided, what would be the urgency of the support ticket? Return priortity as Immediate, High, Medium, Low. Return as Priority: "

In [ ]:
def getTicketCategoryFromModelResponse(CurrentText):
    LLAMA_Response = generateLLAMAPrompt(CurrentText, QueryQuestionAssignPriority, 30, 0.001)
    print(LLAMA_Response)
    colon_index = LLAMA_Response.find(':')
    if colon_index != -1:
        category = LLAMA_Response[colon_index+1:].strip()
        pattern = r'[*":.]'
        category = re.sub(pattern, '', category)
        return category.replace('Category','')
    else:
        return None

In [ ]:
print(getTicketCategoryFromModelResponse(qry))

## Task 4 - Creating a Draft Response

In [ ]:
QueryQuestionDraftResponse = "Based on the information provided, respond back to customer along with estimated time for response from support team"

## Model Output Analysis

## Insights and Recommendations